# Import Packages

In [1]:
!pip install sparqlwrapper

In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def select(query, service='https://query.wikidata.org/sparql'):
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    return pd.json_normalize(result['results']['bindings'])

## 1.1 Olympians

In [4]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel
WHERE
{
  ?person wdt:P3171 ?value .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q1681,en,literal,Abel Mutai
1,uri,http://www.wikidata.org/entity/Q1789,en,literal,Ashton Eaton
2,uri,http://www.wikidata.org/entity/Q1672,en,literal,Dejen Gebremeskel
3,uri,http://www.wikidata.org/entity/Q1712,en,literal,Erick Barrondo
4,uri,http://www.wikidata.org/entity/Q1738,en,literal,Robert Grabarz
...,...,...,...,...,...
29659,uri,http://www.wikidata.org/entity/Q4968493,en,literal,Sara Mustonen
29660,uri,http://www.wikidata.org/entity/Q4350864,en,literal,Mayuko Hagiwara
29661,uri,http://www.wikidata.org/entity/Q2480355,en,literal,Joelle Numainville
29662,uri,http://www.wikidata.org/entity/Q3048753,en,literal,Edwige Pitel


## 1.2 Olympians with Sex or Gender

In [5]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_sex_or_gender.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexgenderLabel.xml:lang,sexgenderLabel.type,sexgenderLabel.value
0,uri,http://www.wikidata.org/entity/Q221185,en,literal,Kateřina Emmons,en,literal,female
1,uri,http://www.wikidata.org/entity/Q222350,en,literal,Piotr Haczek,en,literal,male
2,uri,http://www.wikidata.org/entity/Q225913,en,literal,Miruts Yifter,en,literal,male
3,uri,http://www.wikidata.org/entity/Q225985,en,literal,Eder Sánchez Arraiz,en,literal,male
4,uri,http://www.wikidata.org/entity/Q227338,en,literal,Zsolt Borkai,en,literal,male
...,...,...,...,...,...,...,...,...
29661,uri,http://www.wikidata.org/entity/Q27656433,en,literal,Jesús Cabrera,en,literal,male
29662,uri,http://www.wikidata.org/entity/Q27736817,en,literal,Yair Davidovitz,en,literal,male
29663,uri,http://www.wikidata.org/entity/Q27824142,en,literal,Anthony Coullet,en,literal,male
29664,uri,http://www.wikidata.org/entity/Q27824153,en,literal,Georgi Shikov,en,literal,male


## 1.3 Olympians with Sports

In [6]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(?sportLabel;SEPARATOR=', ') AS ?sports)
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P106 ?sport .
    ?sport rdfs:label ?sportLabel . FILTER(LANG(?sportLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_sports.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sports.type,sports.value
0,uri,http://www.wikidata.org/entity/Q65678,en,literal,Florence Ekpo-Umoh,literal,"athletics competitor, sprinter"
1,uri,http://www.wikidata.org/entity/Q65739,en,literal,Franziska Weber,literal,kayaker
2,uri,http://www.wikidata.org/entity/Q65752,en,literal,Bastian Steger,literal,table tennis player
3,uri,http://www.wikidata.org/entity/Q66014,en,literal,Jörg Fiedler,literal,fencer
4,uri,http://www.wikidata.org/entity/Q66947,en,literal,Anne Cibis,literal,"athletics competitor, sprinter"
...,...,...,...,...,...,...,...
29564,uri,http://www.wikidata.org/entity/Q47306624,en,literal,Heather Hattin,literal,rower
29565,uri,http://www.wikidata.org/entity/Q47307924,en,literal,Colleen Miller,literal,rower
29566,uri,http://www.wikidata.org/entity/Q47341760,en,literal,Tara Laszlo,literal,"short track speed skater, speed skater"
29567,uri,http://www.wikidata.org/entity/Q47345797,en,literal,David Doyle,literal,rower


## 1.4 Olympians with All Demographic Categories

In [7]:
query_string = """
SELECT
  ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(?sportLabel;SEPARATOR=', ') AS ?sports)
WHERE
{
  ?person wdt:P3171 ?value .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  OPTIONAL { ?person wdt:P106 ?sport .
           ?sport rdfs:label ?sportLabel . FILTER(LANG(?sportLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
LIMIT 30000
"""
result=select(query_string)
result.to_csv("Olympians_complete.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexgenderLabel.xml:lang,sexgenderLabel.type,sexgenderLabel.value,sports.type,sports.value
0,uri,http://www.wikidata.org/entity/Q65678,en,literal,Florence Ekpo-Umoh,en,literal,female,literal,"athletics competitor, sprinter"
1,uri,http://www.wikidata.org/entity/Q65739,en,literal,Franziska Weber,en,literal,female,literal,kayaker
2,uri,http://www.wikidata.org/entity/Q65752,en,literal,Bastian Steger,en,literal,male,literal,table tennis player
3,uri,http://www.wikidata.org/entity/Q66014,en,literal,Jörg Fiedler,en,literal,male,literal,fencer
4,uri,http://www.wikidata.org/entity/Q66947,en,literal,Anne Cibis,en,literal,female,literal,"athletics competitor, sprinter"
...,...,...,...,...,...,...,...,...,...,...
29566,uri,http://www.wikidata.org/entity/Q60736546,en,literal,Manuel Gomes,en,literal,male,literal,boxer
29567,uri,http://www.wikidata.org/entity/Q60736915,en,literal,Alex González,en,literal,male,literal,boxer
29568,uri,http://www.wikidata.org/entity/Q60736973,en,literal,János Nagy,en,literal,male,literal,boxer
29569,uri,http://www.wikidata.org/entity/Q60760140,en,literal,Peter Odhiambo,en,literal,male,literal,boxer


## 2.1 People With Medical Conditions

In [8]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,conditions.type,conditions.value
0,uri,http://www.wikidata.org/entity/Q628679,en,literal,Grand Duke Michael Mikhailovich of Russia,literal,influenza
1,uri,http://www.wikidata.org/entity/Q661678,en,literal,Henry Brazeale Walthall,literal,influenza
2,uri,http://www.wikidata.org/entity/Q698151,en,literal,Bohumil Kubišta,literal,"influenza, 1918-1920 flu pandemic"
3,uri,http://www.wikidata.org/entity/Q709077,en,literal,John L. O'Sullivan,literal,influenza
4,uri,http://www.wikidata.org/entity/Q724677,en,literal,William Wilson Hunter,literal,influenza
...,...,...,...,...,...,...,...
11083,uri,http://www.wikidata.org/entity/Q95887988,en,literal,Julie Butler,literal,COVID-19
11084,uri,http://www.wikidata.org/entity/Q95887996,en,literal,Ann Youngerman Smoler,literal,COVID-19
11085,uri,http://www.wikidata.org/entity/Q95887998,en,literal,Thomas J. Waters,literal,COVID-19
11086,uri,http://www.wikidata.org/entity/Q95887999,en,literal,Kyra Michelle Swartz,literal,COVID-19


## 2.2 People With Medical Conditions and Sex or Gender

In [9]:
query_string = """
SELECT 
  ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_sex_or_gender.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexgenderLabel.xml:lang,sexgenderLabel.type,sexgenderLabel.value,conditions.type,conditions.value
0,uri,http://www.wikidata.org/entity/Q628679,en,literal,Grand Duke Michael Mikhailovich of Russia,en,literal,male,literal,influenza
1,uri,http://www.wikidata.org/entity/Q661678,en,literal,Henry Brazeale Walthall,en,literal,male,literal,influenza
2,uri,http://www.wikidata.org/entity/Q698151,en,literal,Bohumil Kubišta,en,literal,male,literal,"influenza, 1918-1920 flu pandemic"
3,uri,http://www.wikidata.org/entity/Q709077,en,literal,John L. O'Sullivan,en,literal,male,literal,influenza
4,uri,http://www.wikidata.org/entity/Q724677,en,literal,William Wilson Hunter,en,literal,male,literal,influenza
...,...,...,...,...,...,...,...,...,...,...
11127,uri,http://www.wikidata.org/entity/Q93394215,en,literal,August Pauger,en,literal,male,literal,COVID-19
11128,uri,http://www.wikidata.org/entity/Q93395406,en,literal,Roman Stadelmann,en,literal,male,literal,COVID-19
11129,uri,http://www.wikidata.org/entity/Q93443209,en,literal,Raquela Mesa-Acosta,en,literal,female,literal,COVID-19
11130,uri,http://www.wikidata.org/entity/Q93449034,en,literal,Barry Webber,en,literal,male,literal,COVID-19


## 2.3 People With Medical Conditions and Cause of Death

In [10]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) (GROUP_CONCAT(DISTINCT ?causedeathLabel;SEPARATOR=', ') AS ?causedeath)
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P509 ?causedeath .
           ?causedeath rdfs:label ?causedeathLabel . FILTER(LANG(?causedeathLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_cause_of_death.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,conditions.type,conditions.value,causedeath.type,causedeath.value
0,uri,http://www.wikidata.org/entity/Q159397,en,literal,Louis Botha,literal,1918-1920 flu pandemic,literal,"influenza, 1918-1920 flu pandemic"
1,uri,http://www.wikidata.org/entity/Q159670,en,literal,"Prince Albert Victor, Duke of Clarence and Avo...",literal,"typhoid fever, pneumonia, influenza",literal,influenza
2,uri,http://www.wikidata.org/entity/Q176368,en,literal,Harold Gilman,literal,"1918-1920 flu pandemic, influenza",literal,"influenza, 1918-1920 flu pandemic"
3,uri,http://www.wikidata.org/entity/Q69045,en,literal,Ernst Albrecht,literal,Alzheimer's disease,literal,Alzheimer's disease
4,uri,http://www.wikidata.org/entity/Q75939,en,literal,Hans Münch,literal,Alzheimer's disease,literal,Alzheimer's disease
...,...,...,...,...,...,...,...,...,...
11083,uri,http://www.wikidata.org/entity/Q43769,en,literal,Ottfried Fischer,literal,"physical disability, Parkinson's disease, obesity",literal,
11084,uri,http://www.wikidata.org/entity/Q44220,en,literal,Anders Behring Breivik,literal,"narcissistic personality disorder, antisocial ...",literal,
11085,uri,http://www.wikidata.org/entity/Q44313,en,literal,Stone Cold Steve Austin,literal,spinal stenosis,literal,
11086,uri,http://www.wikidata.org/entity/Q44378,en,literal,Paul Wight,literal,gigantism,literal,


## 2.4 People With Sexually-Transmitted Infections

In [11]:
query_string = """
SELECT ?person ?personLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
WHERE
{
  ?person wdt:P31 wd:Q5 .
  ?person wdt:P1050 ?condition .
  ?condition wdt:P279 wd:Q12198 .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_STIs.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,conditions.type,conditions.value
0,uri,http://www.wikidata.org/entity/Q15869,en,literal,Freddie Mercury,literal,HIV/AIDS
1,uri,http://www.wikidata.org/entity/Q53325,en,literal,Arthur Ashe,literal,HIV/AIDS
2,uri,http://www.wikidata.org/entity/Q57774,en,literal,Klaus Nomi,literal,HIV/AIDS
3,uri,http://www.wikidata.org/entity/Q77208,en,literal,Horst Bienek,literal,HIV/AIDS
4,uri,http://www.wikidata.org/entity/Q184378,en,literal,Rock Hudson,literal,HIV/AIDS
...,...,...,...,...,...,...,...
138,uri,http://www.wikidata.org/entity/Q107341816,en,literal,Nadia Echazú,literal,HIV/AIDS
139,uri,http://www.wikidata.org/entity/Q107387549,en,literal,Nomandla Yako,literal,HIV/AIDS
140,uri,http://www.wikidata.org/entity/Q109630123,en,literal,William Way,literal,HIV/AIDS
141,uri,http://www.wikidata.org/entity/Q109801653,en,literal,Phil Saviano,literal,HIV/AIDS


## 2.5 People With STIs, Sex or Gender, and Cause of Death

In [12]:
query_string = """
SELECT ?person ?personLabel ?sexgenderLabel (GROUP_CONCAT(DISTINCT ?conditionLabel;SEPARATOR=', ') AS ?conditions) 
  (GROUP_CONCAT(DISTINCT ?causedeathLabel;SEPARATOR=', ') AS ?causedeath)
WHERE
{
  ?person wdt:P31 wd:Q5 .
  OPTIONAL { ?person wdt:P21 ?sexgender . }
  ?person wdt:P1050 ?condition .
  ?condition wdt:P279+ wd:Q12198 .
  ?condition rdfs:label ?conditionLabel . FILTER(LANG(?conditionLabel)='en')
  OPTIONAL { ?person wdt:P509 ?causedeath .
           ?causedeath rdfs:label ?causedeathLabel . FILTER(LANG(?causedeathLabel)='en') }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
GROUP BY ?person ?personLabel ?sexgenderLabel
ORDER BY ?personLabel
"""

result=select(query_string)
result.to_csv("Medical_conditions_STIs_complete.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexgenderLabel.xml:lang,sexgenderLabel.type,sexgenderLabel.value,conditions.type,conditions.value,causedeath.type,causedeath.value
0,uri,http://www.wikidata.org/entity/Q831387,en,literal,Abing,en,literal,male,literal,syphilis,literal,
1,uri,http://www.wikidata.org/entity/Q58640157,en,literal,Adam Johnson,en,literal,male,literal,HIV/AIDS,literal,death from AIDS-related complications
2,uri,http://www.wikidata.org/entity/Q352,en,literal,Adolf Hitler,en,literal,male,literal,syphilis,literal,ballistic trauma
3,uri,http://www.wikidata.org/entity/Q2740907,en,literal,Agustín Gómez-Arcos,en,literal,male,literal,HIV/AIDS,literal,death from AIDS-related complications
4,uri,http://www.wikidata.org/entity/Q80048,en,literal,Al Capone,en,literal,male,literal,"syphilis, neurosyphilis, gonorrhea",literal,bronchopneumonia
...,...,...,...,...,...,...,...,...,...,...,...,...
148,uri,http://www.wikidata.org/entity/Q17627823,en,literal,Víctor Hugo Pérez Peñaloza,en,literal,genderfluid,literal,HIV/AIDS,literal,encephalitis
149,uri,http://www.wikidata.org/entity/Q109630123,en,literal,William Way,en,literal,male,literal,HIV/AIDS,literal,
150,uri,http://www.wikidata.org/entity/Q3570682,en,literal,Xavier Fourcade,en,literal,male,literal,HIV/AIDS,literal,death from AIDS-related complications
151,uri,http://www.wikidata.org/entity/Q3573333,en,literal,Yvan Burger,en,literal,male,literal,HIV/AIDS,literal,death from AIDS-related complications


## 3.1 University of Virginia Employees

In [13]:
query_string = """
SELECT
  ?person ?personLabel
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
}
"""
result=select(query_string)
result.to_csv("UVA_employees.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q1696311,en,literal,Lennart Heimer
1,uri,http://www.wikidata.org/entity/Q1701091,en,literal,John Francis Marchment Middleton
2,uri,http://www.wikidata.org/entity/Q1710863,en,literal,Judah Moshe Eisenberg
3,uri,http://www.wikidata.org/entity/Q1711079,en,literal,Judith Shatin
4,uri,http://www.wikidata.org/entity/Q1729266,en,literal,Karen Parshall
...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q91190793,en,literal,Joseph P. Allen
1689,uri,http://www.wikidata.org/entity/Q91259695,en,literal,Jeffrey W Gander
1690,uri,http://www.wikidata.org/entity/Q91264726,en,literal,Yuichi J Machida
1691,uri,http://www.wikidata.org/entity/Q91281015,en,literal,Jia Liu


## 3.2 UVA Employees with Sex or Gender

In [14]:
query_string = """
SELECT
  ?person ?personLabel ?sexorgender
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P21 ?value .
    ?value rdfs:label ?sexorgender . FILTER(LANG(?sexorgender)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_sex_or_gender.csv")
result

,person.type,person.value,sexorgender.xml:lang,sexorgender.type,sexorgender.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q242625,en,literal,female,en,literal,Mary Ainsworth
1,uri,http://www.wikidata.org/entity/Q275776,en,literal,female,en,literal,Wendy Palmer
2,uri,http://www.wikidata.org/entity/Q289337,en,literal,female,en,literal,Kathryn C. Thornton
3,uri,http://www.wikidata.org/entity/Q445740,en,literal,female,en,literal,Rita Dove
4,uri,http://www.wikidata.org/entity/Q452145,en,literal,female,en,literal,Ann Beattie
...,...,...,...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q109676699,NaN,NaN,NaN,en,literal,Samantha Furr-Rogers
1689,uri,http://www.wikidata.org/entity/Q109689918,NaN,NaN,NaN,en,literal,M. Brittany Johnson
1690,uri,http://www.wikidata.org/entity/Q110937485,NaN,NaN,NaN,en,literal,John R. Hott
1691,uri,http://www.wikidata.org/entity/Q111190908,NaN,NaN,NaN,en,literal,Robert R. Doering


## 3.3 UVA Employees with Ethnicity

In [15]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(DISTINCT ?ethnicityLabel;SEPARATOR=', ') AS ?ethnicity)
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P172 ?ethnicity .
    ?ethnicity rdfs:label ?ethnicityLabel . FILTER(LANG(?ethnicityLabel)='en') }
}
GROUP BY ?person ?personLabel
"""
result=select(query_string)
result.to_csv("UVA_employees_ethnicity.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,ethnicity.type,ethnicity.value
0,uri,http://www.wikidata.org/entity/Q355793,en,literal,James Alan McPherson,literal,African Americans
1,uri,http://www.wikidata.org/entity/Q445740,en,literal,Rita Dove,literal,African Americans
2,uri,http://www.wikidata.org/entity/Q5053606,en,literal,Cato T. Laurencin,literal,African Americans
3,uri,http://www.wikidata.org/entity/Q5248236,en,literal,Deborah E. McDowell,literal,African Americans
4,uri,http://www.wikidata.org/entity/Q6306998,en,literal,Julian Bond,literal,African Americans
...,...,...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q111236330,en,literal,Ravi Kuchimanchi,literal,
1689,uri,http://www.wikidata.org/entity/Q9895,en,literal,Anita K. Jones,literal,
1690,uri,http://www.wikidata.org/entity/Q14101,en,literal,Angela Orebaugh,literal,
1691,uri,http://www.wikidata.org/entity/Q49828,en,literal,Randy Pausch,literal,


## 3.4 UVA Employees with Birthplace

In [16]:
query_string = """
SELECT
  ?person ?personLabel ?birthplace
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P19 ?value . 
    ?value rdfs:label ?birthplace . FILTER(LANG(?birthplace)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_birthplace.csv")
result

,person.type,person.value,birthplace.xml:lang,birthplace.type,birthplace.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q171989,en,literal,New York City,en,literal,James R. Schlesinger
1,uri,http://www.wikidata.org/entity/Q467312,en,literal,New York City,en,literal,Cora Diamond
2,uri,http://www.wikidata.org/entity/Q633049,en,literal,New York City,en,literal,Archibald A. Hill
3,uri,http://www.wikidata.org/entity/Q845573,en,literal,New York City,en,literal,Jonathan Haidt
4,uri,http://www.wikidata.org/entity/Q1362347,en,literal,New York City,en,literal,Randall Thompson
...,...,...,...,...,...,...,...,...
1690,uri,http://www.wikidata.org/entity/Q111051126,NaN,NaN,NaN,en,literal,George Jesse Taylor
1691,uri,http://www.wikidata.org/entity/Q111190908,NaN,NaN,NaN,en,literal,Robert R. Doering
1692,uri,http://www.wikidata.org/entity/Q111195644,NaN,NaN,NaN,en,literal,Anthony J. Beasley
1693,uri,http://www.wikidata.org/entity/Q111236330,NaN,NaN,NaN,en,literal,Ravi Kuchimanchi


## 3.5 UVA Employees with Citizenship

In [17]:
query_string = """
SELECT
  ?person ?personLabel (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship)
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P27 ?citizenship .
    ?citizenship rdfs:label ?citizenshipLabel . FILTER(LANG(?citizenshipLabel)='en') }
}
GROUP BY ?person ?personLabel
"""
result=select(query_string)
result.to_csv("UVA_employees_birthplace.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,citizenship.type,citizenship.value
0,uri,http://www.wikidata.org/entity/Q2559671,en,literal,Samuel Alfred Mitchell,literal,"Canada, United States of America"
1,uri,http://www.wikidata.org/entity/Q3185398,en,literal,Joseph Médard Carrière,literal,Canada
2,uri,http://www.wikidata.org/entity/Q242625,en,literal,Mary Ainsworth,literal,"Canada, United States of America"
3,uri,http://www.wikidata.org/entity/Q731300,en,literal,Ian Stevenson,literal,Canada
4,uri,http://www.wikidata.org/entity/Q7807415,en,literal,Timothy Naftali,literal,Canada
...,...,...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q108545454,en,literal,Peggy P. McNaull,literal,
1689,uri,http://www.wikidata.org/entity/Q108545601,en,literal,Debra G. Perina,literal,
1690,uri,http://www.wikidata.org/entity/Q108545607,en,literal,Eunice M. Singletary,literal,
1691,uri,http://www.wikidata.org/entity/Q108759480,en,literal,Cynthia M. Yoshida,literal,


## 3.6 UVA Employees with Religion

In [18]:
query_string = """
SELECT
  ?person ?personLabel ?religion
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P140 ?value .
    ?value rdfs:label ?religion . FILTER(LANG(?religion)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_religion.csv")
result

,person.type,person.value,religion.xml:lang,religion.type,religion.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q348235,en,literal,Buddhism,en,literal,Trinh Xuan Thuan
1,uri,http://www.wikidata.org/entity/Q1686453,en,literal,Buddhism,en,literal,Jeffrey Hopkins
2,uri,http://www.wikidata.org/entity/Q17143971,en,literal,Buddhism,en,literal,Kurtis Schaeffer
3,uri,http://www.wikidata.org/entity/Q4665815,en,literal,Islam,en,literal,Abdulaziz Sachedina
4,uri,http://www.wikidata.org/entity/Q30323550,en,literal,Judaism,en,literal,Vanessa L. Ochs
...,...,...,...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q111206741,NaN,NaN,NaN,en,literal,John James
1689,uri,http://www.wikidata.org/entity/Q111236330,NaN,NaN,NaN,en,literal,Ravi Kuchimanchi
1690,uri,http://www.wikidata.org/entity/Q9895,NaN,NaN,NaN,en,literal,Anita K. Jones
1691,uri,http://www.wikidata.org/entity/Q14101,NaN,NaN,NaN,en,literal,Angela Orebaugh


## 3.7 UVA Employees with Sexuality

In [19]:
query_string = """
SELECT
  ?person ?personLabel ?sexuality
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P91 ?value4 .
    ?value4 rdfs:label ?sexuality . FILTER(LANG(?sexuality)='en') }
}
"""
result=select(query_string)
result.to_csv("UVA_employees_sexuality.csv")
result

,person.type,person.value,sexuality.xml:lang,sexuality.type,sexuality.value,personLabel.xml:lang,personLabel.type,personLabel.value
0,uri,http://www.wikidata.org/entity/Q7807415,en,literal,homosexuality,en,literal,Timothy Naftali
1,uri,http://www.wikidata.org/entity/Q85747,NaN,NaN,NaN,en,literal,Gernot Zippe
2,uri,http://www.wikidata.org/entity/Q133222,NaN,NaN,NaN,en,literal,Clinton Davisson
3,uri,http://www.wikidata.org/entity/Q164311,NaN,NaN,NaN,en,literal,Kevin Granata
4,uri,http://www.wikidata.org/entity/Q167754,NaN,NaN,NaN,en,literal,Hardy Cross Dillard
...,...,...,...,...,...,...,...,...
1688,uri,http://www.wikidata.org/entity/Q9895,NaN,NaN,NaN,en,literal,Anita K. Jones
1689,uri,http://www.wikidata.org/entity/Q14101,NaN,NaN,NaN,en,literal,Angela Orebaugh
1690,uri,http://www.wikidata.org/entity/Q34296,NaN,NaN,NaN,en,literal,Woodrow Wilson
1691,uri,http://www.wikidata.org/entity/Q49828,NaN,NaN,NaN,en,literal,Randy Pausch


## 3.8 UVA Employees with All Demographic Categories

In [20]:
query_string = """
SELECT
  ?person ?personLabel ?sexorgender (GROUP_CONCAT(DISTINCT ?ethnicityLabel;SEPARATOR=', ') AS ?ethnicity) 
  ?birthplace (GROUP_CONCAT(DISTINCT ?citizenshipLabel;SEPARATOR=', ') AS ?citizenship) ?religion 
  ?sexuality
WHERE
{
  ?person wdt:P108 wd:Q213439 .
  ?person rdfs:label ?personLabel . FILTER(LANG(?personLabel)='en')
  OPTIONAL { ?person wdt:P21 ?value .
           ?value rdfs:label ?sexorgender . FILTER(LANG(?sexorgender)='en') }
  OPTIONAL { ?person wdt:P172 ?ethnicity .
           ?ethnicity rdfs:label ?ethnicityLabel . FILTER(LANG(?ethnicityLabel)='en') }
  OPTIONAL { ?person wdt:P19 ?value2 . 
           ?value2 rdfs:label ?birthplace . FILTER(LANG(?birthplace)='en') }
  OPTIONAL { ?person wdt:P27 ?citizenship .
           ?citizenship rdfs:label ?citizenshipLabel . FILTER(LANG(?citizenshipLabel)='en') }
  OPTIONAL { ?person wdt:P140 ?value3 .
           ?value3 rdfs:label ?religion . FILTER(LANG(?religion)='en') }
  OPTIONAL { ?person wdt:P91 ?value4 .
           ?value4 rdfs:label ?sexuality . FILTER(LANG(?sexuality)='en') }
}
GROUP BY ?person ?personLabel ?sexorgender ?birthplace ?religion ?sexuality
"""

result=select(query_string)
result.to_csv("UVA_employees_complete.csv")
result

,person.type,person.value,personLabel.xml:lang,personLabel.type,personLabel.value,sexorgender.xml:lang,sexorgender.type,sexorgender.value,birthplace.xml:lang,birthplace.type,...,religion.xml:lang,religion.type,religion.value,sexuality.xml:lang,sexuality.type,sexuality.value,ethnicity.type,ethnicity.value,citizenship.type,citizenship.value
0,uri,http://www.wikidata.org/entity/Q7807415,en,literal,Timothy Naftali,en,literal,male,en,literal,...,en,literal,Judaism,en,literal,homosexuality,literal,,literal,Canada
1,uri,http://www.wikidata.org/entity/Q85747,en,literal,Gernot Zippe,en,literal,male,en,literal,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,"Austria, Germany"
2,uri,http://www.wikidata.org/entity/Q133222,en,literal,Clinton Davisson,en,literal,male,en,literal,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,United States of America
3,uri,http://www.wikidata.org/entity/Q164311,en,literal,Kevin Granata,en,literal,male,en,literal,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,United States of America
4,uri,http://www.wikidata.org/entity/Q167754,en,literal,Hardy Cross Dillard,en,literal,male,en,literal,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,uri,http://www.wikidata.org/entity/Q109676699,en,literal,Samantha Furr-Rogers,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,
1691,uri,http://www.wikidata.org/entity/Q109689918,en,literal,M. Brittany Johnson,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,
1692,uri,http://www.wikidata.org/entity/Q109774640,en,literal,Thomas Miller Klubock,en,literal,male,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,United States of America
1693,uri,http://www.wikidata.org/entity/Q109868873,en,literal,James Edmund Dandridge Murdaugh,en,literal,male,en,literal,...,NaN,NaN,NaN,NaN,NaN,NaN,literal,,literal,
